# MAC 5739 - EP 4
Lucas Santana

## Importing libraries

In [133]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [135]:
from typing import List, TypeVar
pandas_type = TypeVar('pandas.core.frame.DataFrame')

In [200]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Input, SpatialDropout2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers import Adam

In [161]:
# from keras.models import Model
# from keras.layers import *
# from keras.optimizers import *
# from keras.losses import *
# from keras.regularizers import *
# from keras.models import Sequential
# from keras.callbacks import *

In [203]:
seed = 42

## Preprocessing

### Importing data

In [26]:
columns_e = ['id_paciente', 'dt_coleta', 'de_origem', 'de_exame', 'de_analito',
       'de_resultado', 'cd_unidade', 'de_valor_referencia']

In [30]:
columns_p = ['id_paciente', 'ic_sexo', 'aa_nascimento', 'cd_pais', 'cd_uf',
       'cd_municipio', 'cd_distrito']

In [54]:
einstein_e = pd.read_csv('./dados/einstein_e.csv')
einstein_e.columns = columns_e
einstein_e['hospital'] = 'einstein'
einstein_e

,id_paciente,dt_coleta,de_origem,de_exame,de_analito,de_resultado,cd_unidade,de_valor_referencia,hospital
0,202018,13/06/2020,HOSP,PCR em tempo real para detecÃ§Ã£o de Coron,Resultado COVID-19:,NÃ£o detectado,NaN,NÃ£o detectado,einstein
1,202025,11/05/2020,HOSP,Sorologia SARS-CoV-2/COVID19 IgG/IgM,IgG: COVID19,0.07,AU/ml,<=0.90,einstein
2,202025,11/05/2020,HOSP,Sorologia SARS-CoV-2/COVID19 IgG/IgM,COVID IgG Interp,NÃ£o reagente,NaN,NaN,einstein
3,202025,11/05/2020,HOSP,PCR em tempo real para detecÃ§Ã£o de Coron,Resultado COVID-19:,NÃ£o detectado,NaN,NÃ£o detectado,einstein
4,202025,11/05/2020,HOSP,Sorologia SARS-CoV-2/COVID19 IgG/IgM,IgM: COVID19,0.33,AU/ml,<=0.90,einstein
...,...,...,...,...,...,...,...,...,...
2062,265360,05/06/2020,HOSP,PCR em tempo real para detecÃ§Ã£o de Coron,Resultado COVID-19:,NÃ£o detectado,NaN,NÃ£o detectado,einstein
2063,265360,05/06/2020,HOSP,Sorologia SARS-CoV-2/COVID19 IgG/IgM,COVID IgM Interp,NÃ£o reagente,NaN,NaN,einstein
2064,265360,05/06/2020,HOSP,Sorologia SARS-CoV-2/COVID19 IgG/IgM,COVID IgG Interp,NÃ£o reagente,NaN,NaN,einstein
2065,265360,05/06/2020,HOSP,Sorologia SARS-CoV-2/COVID19 IgG/IgM,IgM: COVID19,0.77,AU/ml,<=0.90,einstein


In [29]:
einstein_p.columns

Index(['id_paciente', 'ic_sexo', 'aa_nascimento', 'cd_pais', 'cd_uf',
       'cd_municipio', 'cd_distrito'],
      dtype='object')

In [55]:
einstein_p = pd.read_csv('./dados/einstein_p.csv')
einstein_p.columns = columns_p
einstein_p['hospital'] = 'einstein'
einstein_p

,id_paciente,ic_sexo,aa_nascimento,cd_pais,cd_uf,cd_municipio,cd_distrito,hospital
0,204591,M,1954,BR,SP,SAO PAULO,NaN,einstein
1,204990,F,1954,BR,SP,SAO PAULO,NaN,einstein
2,204653,F,1956,BR,SP,SAO PAULO,NaN,einstein
3,202577,F,1957,BR,SP,SAO PAULO,NaN,einstein
4,206125,M,1957,BR,SP,SAO PAULO,NaN,einstein
...,...,...,...,...,...,...,...,...
98,203638,F,2001,BR,SP,SAO PAULO,NaN,einstein
99,205420,F,2009,BR,SP,SAO PAULO,NaN,einstein
100,215350,M,2017,BR,SP,SAO PAULO,NaN,einstein
101,217503,M,2018,BR,SP,SAO PAULO,NaN,einstein


In [56]:
fleury_e = pd.read_csv('./dados/fleury_e.csv')
fleury_e.columns = columns_e
fleury_e['hospital'] = 'fleury'
fleury_e

,id_paciente,dt_coleta,de_origem,de_exame,de_analito,de_resultado,cd_unidade,de_valor_referencia,hospital
0,5B50C7FC316FAFEBF582DB9245E35DB5,02/11/2019,LAB,HEMOGRAMA: sangue total,Hemoglobina,11.9,g/dL,12.0 a 15.5,fleury
1,5B50C7FC316FAFEBF582DB9245E35DB5,02/11/2019,LAB,HEMOGRAMA: sangue total,VCM,81.5,fL,82.0 a 98.0,fleury
2,5B50C7FC316FAFEBF582DB9245E35DB5,02/11/2019,LAB,HEMOGRAMA: sangue total,Leucócitos,5970,/mm3,3.500 a 10.500,fleury
3,5B50C7FC316FAFEBF582DB9245E35DB5,02/11/2019,LAB,HEMOGRAMA: sangue total,Neutrófilos,2360,/mm3,1.700 a 7.000,fleury
4,5B50C7FC316FAFEBF582DB9245E35DB5,02/11/2019,LAB,HEMOGRAMA: sangue total,Eosinófilos,550,/mm3,50 a 500,fleury
...,...,...,...,...,...,...,...,...,...
2046,DE185E011AE5D3A99B3E9DFCC7314840,02/03/2020,LAB,TRANSAMINASE GLUTAMICO-OXALACETICA (TGO),AST (TGO),20,U/L,Até 40,fleury
2047,DE185E011AE5D3A99B3E9DFCC7314840,24/04/2020,LAB,NOVO CORONAVÍRUS 2019 (SARS-CoV-2): DETECÇÃO P...,Covid 19: Detecção por PCR,DETECTADO,NaN,Não detectado,fleury
2048,DE185E011AE5D3A99B3E9DFCC7314840,24/04/2020,LAB,NOVO CORONAVÍRUS 2019 (SARS-CoV-2): DETECÇÃO P...,Covid 19: Material,raspado de nasofaringe,NaN,NaN,fleury
2049,70EC2CFEEEFC4001008937EE46F23F45,24/04/2020,LAB,NOVO CORONAVÍRUS 2019 (SARS-CoV-2): DETECÇÃO P...,Covid 19: Detecção por PCR,NÃO DETECTADO,NaN,Não detectado,fleury


In [57]:
fleury_p = pd.read_csv('./dados/fleury_p.csv')
fleury_p.columns = columns_p
fleury_p['hospital'] = 'fleury'
fleury_p

,id_paciente,ic_sexo,aa_nascimento,cd_pais,cd_uf,cd_municipio,cd_distrito,hospital
0,5B50C7FC316FAFEBF582DB9245E35DB5,F,1982,Brasil,SP,GUARULHOS,NaN,fleury
1,027B6329D6AC1CD1B5DED1DCF87AD204,F,1982,Brasil,SP,SAO PAULO,NaN,fleury
2,ED9B795482AAF5BC59BD819076E922EC,F,1982,Brasil,SP,SAO PAULO,NaN,fleury
3,A9B12C356B595C43ADC11FC38477C8B1,M,1982,Brasil,SP,SAO PAULO,NaN,fleury
4,4AE0B3CE202C44CAE4782752C6ABDA17,F,1982,Brasil,SP,MAUA,NaN,fleury
...,...,...,...,...,...,...,...,...
95,700F3A31468475C2D66A6677D838603D,F,1982,Brasil,SP,SAO PAULO,NaN,fleury
96,77E52BA2720F5104813D1EADE1AF876E,M,1982,Brasil,SP,SAO PAULO,NaN,fleury
97,242457EA77A6B7F6E8A9F34C85BF9793,F,1982,Brasil,SP,SAO PAULO,NaN,fleury
98,DE185E011AE5D3A99B3E9DFCC7314840,M,1982,Brasil,SP,SANTO ANDRE,NaN,fleury


In [88]:
hsl_e = pd.read_csv('./dados/hsl_e.csv')
hsl_e.drop(axis=1, labels='ID_ATENDIMENTO', inplace=True)
hsl_e.columns = columns_e
hsl_e['hospital'] = 'hsl'
hsl_e

,id_paciente,dt_coleta,de_origem,de_exame,de_analito,de_resultado,cd_unidade,de_valor_referencia,hospital
0,B4102D002D0FF995E23072AF92302BEB,26/05/2020 00:00:00,LaboratÃ³rio de Patologia ClÃ­nica,ProteÃ­na C Reativa: plasma,ProteÃ­na C-Reativa,0.06,mg/dL,Ver resultado tradicional,hsl
1,B4102D002D0FF995E23072AF92302BEB,26/05/2020 00:00:00,LaboratÃ³rio de Patologia ClÃ­nica,Tgo,AST (TGO),22,U/L,AtÃ© 40,hsl
2,B4102D002D0FF995E23072AF92302BEB,26/05/2020 00:00:00,LaboratÃ³rio de Patologia ClÃ­nica,Tgp,ALT (TGP),25,U/L,AtÃ© 41,hsl
3,B4102D002D0FF995E23072AF92302BEB,26/05/2020 00:00:00,LaboratÃ³rio de Patologia ClÃ­nica,UrÃ©ia,UrÃ©ia,37,mg/dL,10 a 50,hsl
4,B4102D002D0FF995E23072AF92302BEB,26/05/2020 00:00:00,LaboratÃ³rio de Patologia ClÃ­nica,Calculo da Estimativa da Taxa: de Filtracao Gl...,Creatinina,1.11,mg/dL,0.70 a 1.30,hsl
...,...,...,...,...,...,...,...,...,...
3053,7F6F75D2757F9C6B7A1A8FA6AAA38792,30/03/2020 00:00:00,SaÃºde Corporativa - Fleury - SÃ£o Caetano,COVID-19-PCR para SARS-COV-2: VÃ¡rios Materiai...,CoronavÃ­rus (2019-nCoV),DETECTADO,NaN,NÃ£o detectado,hsl
3054,7F6F75D2757F9C6B7A1A8FA6AAA38792,30/03/2020 00:00:00,SaÃºde Corporativa - Fleury - SÃ£o Caetano,COVID-19-PCR para SARS-COV-2: VÃ¡rios Materiai...,Material (2019-nCoV),swab de nasofaringe,NaN,NaN,hsl
3055,7F6F75D2757F9C6B7A1A8FA6AAA38792,30/03/2020 00:00:00,SaÃºde Corporativa - Fleury - SÃ£o Caetano,AntÃ­geno: Influenza: Teste RÃ¡pido (AGINFLU),Influenza A e B - teste rÃ¡pido,NEGATIVO,ng/mL,Negativo,hsl
3056,CF58DD8870CB8AD13D5889B62F19C79A,30/03/2020 00:00:00,SaÃºde Corporativa - Fleury - SÃ£o Caetano,COVID-19-PCR para SARS-COV-2: VÃ¡rios Materiai...,CoronavÃ­rus (2019-nCoV),DETECTADO,NaN,NÃ£o detectado,hsl


In [89]:
hsl_p = pd.read_csv('./dados/hsl_p.csv')
hsl_p.columns = columns_p
hsl_p['hospital'] = 'hsl'
hsl_p

,id_paciente,ic_sexo,aa_nascimento,cd_pais,cd_uf,cd_municipio,cd_distrito,hospital
0,7D45FE3D655E66784DD28970EF49ADCA,F,1984,Brasil,SP,SÃ£o Paulo,NaN,hsl
1,E33BF02768C6B38404CB6533703D0D2B,M,1984,Brasil,SP,SÃ£o Paulo,NaN,hsl
2,0BA8C352CCBDC9BEC24A30D5938E49B2,M,1987,Brasil,SP,Sao Paulo,NaN,hsl
3,C7A2FFAF332245084105ABBC61C744E9,M,1980,Brasil,SP,Guarulhos,NaN,hsl
4,3ECDE1017B0467B9D6C133C9770D99EA,F,1982,Brasil,SP,SÃ£o Paulo,NaN,hsl
...,...,...,...,...,...,...,...,...
95,D3202DB91A6C3480017591827AC64B1E,M,1981,Brasil,SP,SÃ£o Paulo,NaN,hsl
96,1CFD4DF25719D30AF7701D93C1F04306,F,1986,Brasil,SP,SÃ£o Paulo,NaN,hsl
97,3BDA24CD5435BE2ACF757CE52CB37CA3,F,1984,Brasil,SP,SÃ£o Paulo,NaN,hsl
98,D8A21F43AF43240A4437314500570E24,M,1990,Brasil,SP,Sao Paulo,NaN,hsl


### Concat dataframes

In [139]:
list_e_df = [einstein_e, fleury_e, hsl_e]
list_p_df = [einstein_p, fleury_p, hsl_p]

In [140]:
df_e = pd.concat(list_e_df)
df_p = pd.concat(list_p_df)

In [141]:
df_e

,id_paciente,dt_coleta,de_origem,de_exame,de_analito,de_resultado,cd_unidade,de_valor_referencia,hospital
0,202018,13/06/2020,HOSP,PCR em tempo real para detecÃ§Ã£o de Coron,Resultado COVID-19:,NÃ£o detectado,NaN,NÃ£o detectado,einstein
1,202025,11/05/2020,HOSP,Sorologia SARS-CoV-2/COVID19 IgG/IgM,IgG: COVID19,0.07,AU/ml,<=0.90,einstein
2,202025,11/05/2020,HOSP,Sorologia SARS-CoV-2/COVID19 IgG/IgM,COVID IgG Interp,NÃ£o reagente,NaN,NaN,einstein
3,202025,11/05/2020,HOSP,PCR em tempo real para detecÃ§Ã£o de Coron,Resultado COVID-19:,NÃ£o detectado,NaN,NÃ£o detectado,einstein
4,202025,11/05/2020,HOSP,Sorologia SARS-CoV-2/COVID19 IgG/IgM,IgM: COVID19,0.33,AU/ml,<=0.90,einstein
...,...,...,...,...,...,...,...,...,...
3053,7F6F75D2757F9C6B7A1A8FA6AAA38792,30/03/2020 00:00:00,SaÃºde Corporativa - Fleury - SÃ£o Caetano,COVID-19-PCR para SARS-COV-2: VÃ¡rios Materiai...,CoronavÃ­rus (2019-nCoV),DETECTADO,NaN,NÃ£o detectado,hsl
3054,7F6F75D2757F9C6B7A1A8FA6AAA38792,30/03/2020 00:00:00,SaÃºde Corporativa - Fleury - SÃ£o Caetano,COVID-19-PCR para SARS-COV-2: VÃ¡rios Materiai...,Material (2019-nCoV),swab de nasofaringe,NaN,NaN,hsl
3055,7F6F75D2757F9C6B7A1A8FA6AAA38792,30/03/2020 00:00:00,SaÃºde Corporativa - Fleury - SÃ£o Caetano,AntÃ­geno: Influenza: Teste RÃ¡pido (AGINFLU),Influenza A e B - teste rÃ¡pido,NEGATIVO,ng/mL,Negativo,hsl
3056,CF58DD8870CB8AD13D5889B62F19C79A,30/03/2020 00:00:00,SaÃºde Corporativa - Fleury - SÃ£o Caetano,COVID-19-PCR para SARS-COV-2: VÃ¡rios Materiai...,CoronavÃ­rus (2019-nCoV),DETECTADO,NaN,NÃ£o detectado,hsl


### Understanding data

### Cleaning data

In [130]:
regex_list = [
    r'<.*?>',
    r'<code>(.|(\n))*?</code>',
    r'[^ \w+]',
    r'\d+',
    r'\s+'
]
substituation_list = [
    '',
    '',
    ' ',
    ' ',
    ' '
]

In [131]:
def clean_column(
    df: pandas_type,
    dirty_column: str,
    cleaned_column: str,
    regex_list: List[str],
    substituation_list: List[str]) -> pandas_type:
    
    punctuation_regex = re.compile(r'[^ \w]', 2)
    spaces_regex = re.compile(r'\s+', 2)
    
    df[cleaned_column] = df[cleaned_column].apply(lambda x : punctuation_regex.sub(' ', x))
    df[cleaned_column] = df[cleaned_column].apply(lambda x : x.lower())
    df[cleaned_column] = df[cleaned_column].apply(lambda x : spaces_regex.sub(' ', x))
    
    return df

## Neural Network

### Spliting data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.20, random_state = seed)

### Architecture

In [164]:
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)

In [192]:
input_ = Input((256, 256,1))
x = input_
x = Conv2D(32, kernel_size = (3,3), strides = (1, 1), padding = 'same', activation = 'relu')(x)
x = Conv2D(32, kernel_size = (3,3), strides = (1, 1), padding = 'same', activation = 'relu')(x)
x = Conv2D(32, kernel_size = (3,3), strides = (1, 1), padding = 'same', activation = 'relu')(x)
x = SpatialDropout2D(0.1)(x)
x = MaxPooling2D(2, 2)(x)
# 8x8
#x = GlobalMaxPooling2D()(x)
x = Flatten()(x)
x = Dense(100, activation  =  'relu')(x)
#x = Dropout(0.1)(x)
x = Dense(1, activation  =  'sigmoid')(x)

In [194]:
model = Model(input_, x)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 256, 256, 1)]     0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 256, 256, 32)      320       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 256, 256, 32)      9248      
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 256, 256, 32)      9248      
_________________________________________________________________
spatial_dropout2d_7 (Spatial (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 524288)            0   

### Parâmetros de Otimização

In [201]:
batch_size = 64
num_epochs = 100
learning_rate = 1e-4

weights_filepath = "weights.h5"
callbacks = [
    ModelCheckpoint(weights_filepath, monitor='val_acc', mode='max',
                             verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_acc', mode='max', patience=10),
    TensorBoard(log_dir='./logs/logs')
]

model.compile(loss = 'binary_crossentropy',
              optimizer = Adam(lr = learning_rate), metrics = ['acc'])

### Training

In [202]:
history = model.fit(X_train, y_train,
          batch_size = batch_size,
          epochs = num_epochs,
          verbose = 1,
          validation_data = (X_val, y_val),
          callbacks=callbacks)
model.load_weights(weights_filepath)

NameError: name 'X_train' is not defined

### Ploting error and accuracy graphs

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.ylabel('acurácia')
plt.xlabel('época')
plt.legend(['treino', 'validação'], loc = 'upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('época')
plt.legend(['treino', 'validação'], loc = 'upper left')
plt.show()

### Evaluating results

In [ ]:
model.evaluate(X_test, y_test)[1]